# Retrieve grapg info and evaluate results across different models

In [1]:
# from transformers import pipeline

# ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all")
# text = "Patient John Doe was diagnosed with diabetes at Mayo Clinic."
# entities = ner_pipeline(text)

# print entities only

In [2]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index, generate_response, run_trial, create_md

# from lib import create_vector_index, generate_response, run_trial,create_md
from dotenv import load_dotenv
import pandas as pd

# from sentence_transformers import SentenceTransformer

# Force reload of the .env file
load_dotenv(override=True)

/Users/zhangbobby/Desktop/Capstone/GrapgRAG_experiments/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

## create vector index

In [4]:
create_vector_index(graph, "entities")

In [6]:
# Question = "Among critically ill patients in septic shock, does low-dose hydrocortisone therapy improve survival?"
Question = "In ARDS, how does early neuromuscular blockage improve mortality?"
response, _ = generate_response(graph, Question)

In [11]:
# Method 2: Print detailed response information
print("\n=== Detailed Response Info ===")
print(f"Model used: {response.model}")
print(f"Response content: {response.choices[0].message.content}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(f"Prompt tokens: {response.usage.prompt_tokens}")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")



=== Detailed Response Info ===
Model used: gpt-4-turbo-2024-04-09
Response content: ["ACURASYS.pdf"]
Finish reason: stop
Prompt tokens: 2069
Completion tokens: 8
Total tokens: 2077


In [ ]:
print("Context:", _)

Context:  ARDS DIAGNOSIS_TO_INCLUSION_TIME 16 hours (interquartile range, 6 to 29),
 ARDS STUDIED_IN ACURASYS study,
 ARDS STUDIED_IN n engl j med,
 ARDS STUDIED_IN n engl j med, n engl j med PUBLISHED_IN High-Frequency Oscillation,
 ARDS STUDIED_IN n engl j med, n engl j med PUBLISHED_ON september 16, 2010,
 ARDS ONSET_TO enrollment,
 ARDS ONSET_OBSERVED_IN patients in our trial,
 ARDS HAS_SEVERITY moderate-to-severe,
 Neuromuscular Blocking Agents USED_FOR ARDS,
 Neuromuscular Blocking Agents USED_FOR ARDS, Neuromuscular Blocking Agents HAS_RISK muscle weakness,
 Neuromuscular Blocking Agents USED_FOR ARDS, Neuromuscular Blocking Agents HAS_EFFECT allowing for the accurate adjustment of tidal volume and pressure levels,
 Neuromuscular Blocking Agents USED_FOR ARDS, Neuromuscular Blocking Agents HAS_EFFECT limiting the risk of asynchrony-related alveolar collapse,
 Neuromuscular Blocking Agents USED_FOR ARDS, Neuromuscular Blocking Agents HAS_EFFECT decrease in lung or systemic inflam

In [ ]:
print(context)

NameError: name 'context' is not defined

In [ ]:
Questions = [
    "Among patients with severe sepsis or septic shock, does administration of albumin 20% solution to maintain serum albumin level ≥3 g/dL (≥30 g/L) reduce all-cause mortality at 28 days when compared to no albumin?",
    "Among patients with septic shock and relative adrenal insufficiency, do corticosteroids reduce 28-day mortality?",
    "In adult patients with septic shock, does low-dose hydrocortisone plus fludrocortisone for 7 days affect overall survival at 90 days?",
    "In patients presenting to an emergency department with severe sepsis or septic shock, does early goal-directed therapy reduce all-cause mortality at 90 days when compared to usual care?",
    "Among critically ill patients in septic shock, does low-dose hydrocortisone therapy improve survival?",
]

In [ ]:
output_df = run_trial(graph, Questions)
output_df

[{'node.id': 'albumin', 'score': 0.9995894432067871}]
[{'node.id': 'severe sepsis', 'score': 0.9609296321868896}]
[{'node.id': 'shock', 'score': 0.9963037967681885}]
[{'node.id': 'The condition', 'score': 0.713494062423706}]
[{'node.id': 'seven days', 'score': 0.8874611854553223}]
[{'node.id': 'Increased mortality', 'score': 0.9090418815612793}]
[{'node.id': 'albumin', 'score': 0.9995894432067871}]
[{'node.id': 'severe sepsis', 'score': 0.9609296321868896}]
[{'node.id': 'shock', 'score': 0.9963037967681885}]
[{'node.id': 'seven days', 'score': 0.8874611854553223}]
[{'node.id': 'Corticosteroids', 'score': 0.9996142387390137}]
[{'node.id': 'Increased mortality', 'score': 0.9090418815612793}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]
[{'node.id': 'Corticosteroids', 'score': 0.9996142387390137}]
[{'node.id': 'adrenal insufficiency', 'score': 0.999896764755249}]
[{'node.id': 'patients', 'score': 0.8938620090484619}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]

,llama3.1:latest-None,llama3.1:latest-vector,llama3.1:latest-graph,llama3.1:latest-hybrid
Question 1,"[Based on the context you provided, which is n...","[Based on the provided context, I cannot deter...",[It looks like you've provided a large block o...,[It appears you have a large dataset of statem...
Question 2,"[Based on the available evidence, corticostero...","[Based on the provided context, I can identify...",[A very specific and interesting question!\n\n...,[It appears that you have provided a list of m...
Question 3,[A great question!\n\nBased on the available e...,"[Based on the provided context [(File Name, Te...",[It seems like you've provided a large amount ...,[This appears to be a collection of text fragm...
Question 4,[Based on the provided context (which isn't ex...,"[Based on the provided context, I can answer y...",[It looks like you've provided a large amount ...,[It appears that you have a large dataset rela...
Question 5,"[Based on the provided context, I can provide ...","[Based on the provided context, it appears tha...",[It looks like you've provided a large block o...,[This appears to be a collection of text snipp...


In [ ]:
# write to csv
output_df.to_csv("trial1.csv", index=False)
output_df["llama3.1:latest-hybrid"][2]

/var/folders/q4/0d5y6tc142n0r8342v4l5tj40000gn/T/ipykernel_84097/921738572.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output_df["llama3.1:latest-hybrid"][2]


['This appears to be a collection of text fragments related to septic shock and the use of hydrocortisone and fludrocortisone as treatments. There are several key points that can be extracted from these texts:\n\n1. **Study Context**: The study did not include fludrocortisone, but its results showed a 32% death rate at 28 days in the placebo group.\n2. **Use of Fludrocortisone**: It is used for septic shock and has been studied in the context of adrenal insufficiency and glucocorticoid therapy.\n3. **Septic Shock Defined**: Septic shock is defined as the presence of a clinically or microbiologically documented condition associated with cortisol response to corticotrophin, cortisol levels, hydrocortisone, acute respiratory distress syndrome, severe sepsis, and neuromuscular weakness.\n4. **Treatment for Septic Shock**: Hydrocortisone plus fludrocortisone is used as an adjunctive glucocorticoid therapy for septic shock.\n5. **Study Results**: The APROCCHSS trial showed a survival benefit

## Trial Setup

In [ ]:
output_path = "trial_outputs/"
for i in range(10):
    print(f"Running trial {i}")
    output_df = run_trial(graph, Questions)
    output_df.to_csv(output_path + f"trial{i}.csv", index=False)
    create_md(
        output_path + f"trial{i}.csv", output_path + f"trial{i}.md", questions=Questions
    )

Running trial 0
[{'node.id': 'severe sepsis', 'score': 0.9998965263366699}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]
[{'node.id': 'albumin', 'score': 0.9995894432067871}]
[{'node.id': '20% albumin solution', 'score': 0.8572235107421875}]
[{'node.id': '11 patients', 'score': 0.6312956809997559}]
[{'node.id': 'G.H.G.', 'score': 0.7848110198974609}]
[{'node.id': 'G.H.G.', 'score': 0.7887003421783447}]
[{'node.id': 'seven days', 'score': 0.8874611854553223}]
[{'node.id': 'severe sepsis', 'score': 0.9998965263366699}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]
[{'node.id': 'albumin', 'score': 0.9995894432067871}]
[{'node.id': 'The condition', 'score': 0.713494062423706}]
[{'node.id': '11 patients', 'score': 0.6312956809997559}]
[{'node.id': 'G.H.G.', 'score': 0.7848110198974609}]
[{'node.id': 'G.H.G.', 'score': 0.7887003421783447}]
[{'node.id': 'seven days', 'score': 0.8874611854553223}]
[{'node.id': 'Corticosteroids', 'score': 0.9996142387390137}]
[{'node.

[#DBF0]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.28.184.63', 7687)) (ResolvedIPv4Address(('34.28.184.63', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.0021246277673463s (Unable to retrieve routing information)


[{'node.id': 'adults', 'score': 0.916346549987793}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]
[{'node.id': 'Hydrocortisone', 'score': 0.9999010562896729}]
[{'node.id': 'Fludrocortisone', 'score': 0.9997668266296387}]
[{'node.id': 'adults', 'score': 0.916346549987793}]
[{'node.id': 'patients', 'score': 0.9984817504882812}]
[{'node.id': 'Septic shock', 'score': 0.9999065399169922}]
[{'node.id': 'Hydrocortisone', 'score': 0.9999010562896729}]
[{'node.id': 'Fludrocortisone', 'score': 0.9997668266296387}]
[{'node.id': 'seven days', 'score': 0.8874611854553223}]
[{'node.id': 'emergency department', 'score': 0.9998016357421875}]
[{'node.id': 'severe sepsis', 'score': 0.9609296321868896}]
[{'node.id': 'shock', 'score': 0.9963037967681885}]
[{'node.id': 'supportive therapy', 'score': 0.9205706119537354}]
[{'node.id': 'Increased mortality', 'score': 0.9090418815612793}]
[{'node.id': 'emergency department', 'score': 0.9998016357421875}]
[{'node.id': 'severe sepsis', 'score': 0.99